In [ ]:
# LangChain: Memory

## Connect to OpenAI and define our model

Create an OpenAI API key for this project at [https://platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys). 

Copy `.env.sample` to `.env` and replace `<YOUR_OPENAI_API_KEY>` with the one you created.

Let's get started!

Let's make sure that we can read our OpenAI key from the `.env` file we created.

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Specify our model
llm_model="gpt-3.5-turbo-1106"

## ConversationBufferMemory

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [4]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

/Users/rob/repos/deeplearning-ai/langchain-for-llm-application-development/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


### Let's explore using conversational memory.

In [5]:
conversation.predict(input="Hi, my name is Rob")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Rob
AI:

> Finished chain.


"Hello Rob! It's nice to meet you. My name is AI and I'm here to help you with any questions or information you need. How can I assist you today?"

In [6]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Rob
AI: Hello Rob! It's nice to meet you. My name is AI and I'm here to help you with any questions or information you need. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


"1+1 equals 2. Is there anything else you'd like to know?"

In [7]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Rob
AI: Hello Rob! It's nice to meet you. My name is AI and I'm here to help you with any questions or information you need. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you'd like to know?
Human: What is my name?
AI:

> Finished chain.


"Your name is Rob. Is there anything else you'd like to know, Rob?"

In [8]:
print(memory.buffer)

Human: Hi, my name is Rob
AI: Hello Rob! It's nice to meet you. My name is AI and I'm here to help you with any questions or information you need. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you'd like to know?
Human: What is my name?
AI: Your name is Rob. Is there anything else you'd like to know, Rob?


In [9]:
# The curly braces represent an empty dictionary
memory.load_memory_variables({})


{'history': "Human: Hi, my name is Rob\nAI: Hello Rob! It's nice to meet you. My name is AI and I'm here to help you with any questions or information you need. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 equals 2. Is there anything else you'd like to know?\nHuman: What is my name?\nAI: Your name is Rob. Is there anything else you'd like to know, Rob?"}

### Let's add a few things to memory explicitly

In [10]:
memory = ConversationBufferMemory()
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})
print(memory.buffer)

Human: Hi
AI: What's up


### We can continue to add additional data to our memory by saving context

In [11]:
memory.load_memory_variables({})
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [12]:
from langchain.memory import ConversationBufferWindowMemory

# The window size is 1, so only the last context is saved from the human and from the chatbot
memory = ConversationBufferWindowMemory(k=1)               

# Example context saved to memory
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

# Our memory only contains the last saved context
memory.load_memory_variables({})
# {'history': 'Human: Not much, just hanging\nAI: Cool'}

{'history': 'Human: Not much, just hanging\nAI: Cool'}

### EXAMPLE: If we apply our last conversation to this type of buffer window memory, note that our LLM will not have an idea of our name

In [14]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

conversation.predict(input="Hi, my name is Rob")

"Hello Rob! It's nice to meet you. My name is AI and I'm here to help you with any questions or tasks you have. How can I assist you today?"

In [15]:
conversation.predict(input="What is 1+1?")

"1+1 equals 2. Is there anything else you'd like to know?"

In [16]:
conversation.predict(input="What is my name?")

"I'm sorry, I don't have access to your personal information, so I don't know your name. Is there anything else I can help you with?"

### EXAMPLE: Re-run the conversation with verbose set to True

In [17]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

conversation.predict(input="Hi, my name is Rob")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Rob
AI:

> Finished chain.


"Hello Rob! It's nice to meet you. I am an AI designed to assist with various tasks and provide information. How can I help you today?"

In [18]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Rob
AI: Hello Rob! It's nice to meet you. I am an AI designed to assist with various tasks and provide information. How can I help you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 equals 2. Is there anything else you would like to know?'

In [19]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


"I'm sorry, I don't have access to your personal information, so I don't know your name. Is there anything else I can help you with?"

## ConversationTokenBufferMemory

Before we begin, let's install the following dependencies from the command-line of our Python virtual environment:

```sh
(.venv) % pip install tiktoken
(.venv) % pip freeze > requirements.txt
```

In [20]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [27]:
# TODO: Experiment with different token limits to see how it affects the memory.load_memory_variables() output
MAX_TOKEN_LIMIT=50

# This approach will restrict memory to a maximum number of tokens
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=MAX_TOKEN_LIMIT)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

memory.load_memory_variables({})

{'history': 'AI: Charming!'}

## ConversationSummaryMemory

In [30]:
# Instead of storing the entire long conversation and restrict it to a max token limit, we can have the LLM summarize the conversation so far - and then limit that to the max token limit
from langchain.memory import ConversationSummaryBufferMemory

# TODO: Experiment with different token limits to see how it affects the memory.load_memory_variables() output
# For example, a max token limit of 400 will retain the entire conversation so far, but a limit of 100 will store a summary of the conversation
MAX_TOKEN_LIMIT=100

# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=MAX_TOKEN_LIMIT)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer to discuss the latest in AI.'}

In [31]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer to discuss the latest in AI.
Human: What would be a good demo to show?
AI:

> Finished chain.


{'history': "System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer to discuss the latest in AI. The human asks what would be a good demo to show.\nAI: We could demonstrate the latest natural language processing capabilities of LangChain, including its ability to analyze and summarize large volumes of text data in real-time. We could also showcase the new AI-powered recommendation engine that we've been working on, which can provide personalized product recommendations based on user preferences and behavior. Additionally, we could highlight the improvements we've made to the chatbot interface, which now has enhanced conversational abilities and can handle more complex user queries."}